In [1]:
set more off

est clear

quietly use "/Users/Charlie/Dropbox/asymmetry by Design/Data/asymmetry_20191229.dta", clear

quietly collapse (rawsum) sftesale sftetotl (mean) all_under fed_grant_pct_w system_under grad_rate_150_p_w ///
pctchrtgrbkaat_w pctchrtgrwhitt_w pctchrtgrhispt_w loan_amount_borrower_c_w tuitionall_c_w selective ///
white_share_w black_share_w hisp_share_w pell_grants_per_fte_c_w dist_pell_s ///
dpcttype_health dpcttype_law dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz online ///
(first) state_n   (min) iclevel (max) law_enf_frst_this_yr alt2multi_brands_all pe_po [aweight=all_under], by(systemid year)

quietly gen fracsales2= sftesale /sftetotl * 100

quietly label var fracsales2 "% employees sales"

quietly replace system_under =system_under/1000

quietly eststo: reg fracsales2 alt2multi_brands_all pe_po system_under  online selective ib3.iclevel ///
fed_grant_pct_w white_share_w black_share_w hisp_share_w dpcttype* i.year i.state_n, cluster(systemid)

In [ ]:
quietly use "/Users/Charlie/Dropbox/asymmetry by Design/Data/asymmetry_20191229.dta", clear

foreach var in tuitionall_c_w loan_amount_borrower_c_w ft_faculty_per_100fte_w { 
    
quietly eststo: reg `var' alt2multi_brands_all pe_po system_under all_under online selective ib3.iclevel ///
fed_grant_pct_w white_share_w black_share_w hisp_share_w dpcttype* i.year i.state_n, cluster(unitid)

}
quietly label var fed_grant_pct_w "% low-income"
quietly label var fracsales2 "% employees sales"

esttab using t3a_asymmetryols20200912.rtf, title(Table 3A: OLS estimates for predatory practices) ///
 keep(alt2multi_brands_all pe_po system_under all_under online selective 1.iclevel 2.iclevel ///
fed_grant_pct_w white_share_w black_share_w hisp_share_w dpcttype_health dpcttype_law ///
dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz _cons)  ///
stats(r2 r2_p N_clust N, fmt(2 2 %9.0fc %9.0fc) label("r2" "pseudo r2" "school/firm clusters" N)) ///
se(%9.1fc) b(%9.1fc) nogaps  ///
 star(* 0.05 ** 0.01 *** 0.001) label varwidth(10) replace

In [ ]:
%html
esttab, ///
 keep(alt2multi_brands_all pe_po system_under all_under online selective 1.iclevel 2.iclevel ///
fed_grant_pct_w white_share_w black_share_w hisp_share_w dpcttype_health dpcttype_law ///
dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz _cons) order(alt2multi_brands_all pe_po system_under all_under) ///
stats(r2 r2_p N_clust N, fmt(2 2 %9.0fc %9.0fc) label("r2" "pseudo r2" "school/firm clusters" N)) ///
se(%9.1fc) b(%9.1fc) nogaps ///
 star(* 0.05 ** 0.01 *** 0.001) html label varwidth(10)

In [ ]:
quietly est clear

foreach var in grad_rate_150_p4yr_w grad_rate_150_p2yr_w  { 
    
quietly eststo: reg `var' alt2multi_brands_all pe_po system_under all_under online selective ib3.iclevel ///
fed_grant_pct_w white_share_w black_share_w hisp_share_w dpcttype* i.year i.state_n, cluster(unitid)

}

quietly use "/Users/Charlie/Dropbox/asymmetry by Design/Data/d_OPEID_asymmetry.dta", clear
quietly gen pe_po=0
quietly replace pe_po=1 if pe_ind==1 | public==1

quietly by opeid: gen l2pe_po=pe_po[_n-2]
quietly by opeid: gen l2alt2multi_brands_all=alt2multi_brands_all[_n-2]

quietly gen pe_nolag=pe_po
quietly gen alt2nolag=alt2multi_brands_all

quietly replace alt2multi_brands_all=l2alt2multi_brands_all
quietly replace pe_po=l2pe_po

quietly gen fed_grant_unitid=fed_grant_pct_w

quietly replace fed_grant_pct_w=rpy_3yr_pct_lo

quietly eststo: reg rpy_3yr_rt_supp_w alt2multi_brands_all pe_po ///
all_under system_under online selective ib3.iclevel fed_grant_pct_w ///
white_share_w black_share_w hisp_share_w dpcttype* i.year i.state_n if rankopeid==1, cluster(opeid)

quietly replace fed_grant_pct_w=earn_pct_lo

quietly eststo: reg mn_earn_wne_p6_c_w alt2multi_brands_all pe_po ///
all_under system_under online selective ib3.iclevel fed_grant_pct_w ///
white_share_w black_share_w hisp_share_w dpcttype* i.year i.state_n if rankopeid==1, cluster(opeid)

quietly replace fed_grant_pct_w=fed_grant_unitid

quietly replace alt2multi_brands_all=alt2nolag
quietly replace pe_po=pe_nolag

quietly sort opeid year
quietly by opeid: replace fed_grant_pct_w=fed_grant_pct_w[_n-1] if year==2016

quietly gen borrower_defense_100student = claimcount / fte_count * 100
winsor borrower_defense_100student, p(.01) gen(borrower_defense_100student_w)

quietly eststo: poisson claimcount alt2multi_brands_all pe_po ///
all_under system_under online selective ib3.iclevel ///
white_share_w black_share_w hisp_share_w  dpcttype* ///
fed_grant_pct_w i.state_n, rob noomitted

In [ ]:
quietly {
    quietly use "/Users/Charlie/Dropbox/asymmetry by Design/Data/asymmetry_20191229.dta", clear

quietly collapse (rawsum) sftesale sftetotl (mean) fed_grant_pct_w system_under grad_rate_150_p_w ///
pctchrtgrbkaat_w pctchrtgrwhitt_w pctchrtgrhispt_w loan_amount_borrower_c_w tuitionall_c_w selective ///
white_share_w black_share_w hisp_share_w pell_grants_per_fte_c_w dist_pell_s ft_faculty_per_100fte_w ///
dpcttype_health dpcttype_law dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz online all_under ///
(first) state_n   (min) iclevel (max) law_enf_frst_this_yr alt2multi_brands_all pe_po ///
grad_rate_150_p4yr_w grad_rate_150_p2yr_w [aweight=all_under], by(systemid year)

 eststo: probit law_enf_frst_this_yr alt2multi_brands_all pe_po system_under online selective i.iclevel ///
fed_grant_pct_w white_share_w black_share_w hisp_share_w dpcttype* i.state_n i.year, cluster(systemid)

gen claimcount=.
gen mn_earn_wne_p6_c_w=.
gen rpy_3yr_rt_supp_w=.
gen earn_pct_lo=.
gen fracsales2=.
quietly label var mn_earn_wne_p6_c_w "earnings"
quietly label var rpy_3yr_rt_supp_w "repayment %"
    gen claimcount=.
quietly label var claimcount "fraud claims"
quietly label var earn_pct_lo "% low-income"
label var alt2multi_brands_all "multi-brand"
label var pe_po "investor owned"
label var system_under "firm-level enrollment (thousands)"
label var all_under "campus enrollment (thousands)"
label var online "online"
label var selective "selective admissions"
label var fed_grant_pct_w "% low-income"
    label var fracsales2 "% employees sales"
    label drop iclevel
label define iclevel 1 "BA offered" 2 "AA offered" 
    label values iclevel iclevel
label var black_share_w "% Black"
label var hisp_share_w "% Hispanic"
label var white_share_w "% White"
label var dpcttype_arts "% degrees art"
label var dpcttype_biz "% degrees biz"
label var dpcttype_culinarycosmetic "% degrees personal services"
label var dpcttype_health "% degrees health"
label var dpcttype_law "% degrees law (undergrad)"
label var dpcttype_tech "% degrees tech"
label var tuitionall_c_w tuition
label var loan_amount_borrower_c_w borrowing
label var ft_faculty_per_100fte_w "# faculty"
label var law_enf_frst_this_yr "law enforcement"
label var grad_rate_150_p4yr_w "BA grad rate"
label var grad_rate_150_p2yr_w "other grad rate"
}

In [ ]:
%html
esttab, ///
 keep(alt2multi_brands_all pe_po system_under all_under online selective 1.iclevel 2.iclevel ///
fed_grant_pct_w white_share_w black_share_w hisp_share_w dpcttype_health dpcttype_law ///
dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz _cons)  ///
stats(r2 r2_p N_clust N, fmt(2 2 %9.0fc %9.0fc) label("r2" "pseudo r2" "school/firm clusters" N)) ///
se(%9.1fc) b(%9.1fc) nogaps stardetach eqlabels("") ///
 star(* 0.05 ** 0.01 *** 0.001) html label varwidth(10)

In [ ]:
esttab using t3b_asymmetryols20200912.rtf, title(Table 3B: OLS and MLE estimates for student outcomes and legal actions) ///
 keep(alt2multi_brands_all pe_po system_under all_under online selective 1.iclevel 2.iclevel ///
fed_grant_pct_w white_share_w black_share_w hisp_share_w dpcttype_health dpcttype_law ///
dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz _cons)  ///
stats(r2 r2_p N_clust N, fmt(2 2 %9.0fc %9.0fc) label("r2" "pseudo r2" "school/firm clusters" N)) ///
se(%9.1fc) b(%9.1fc) nogaps eqlabels("") ///
 star(* 0.05 ** 0.01 *** 0.001) label varwidth(10) replace